In [29]:
import numpy as np
import pandas as pd


import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.naive_bayes import MultinomialNB

from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os

import warnings
warnings.filterwarnings('ignore')

In [30]:
#importing the training data
imdb_data=pd.read_csv('IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [31]:
#sentiment count
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [32]:
#Summary of the dataset
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [33]:
#split the dataset
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [34]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Text normalization

In [35]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [36]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

Removing special characters

In [37]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

Text stemming

In [38]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [39]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'again', 'isn', 'she', 'here', 'during', 'before', 't', 'all', 'over', "needn't", 'an', 'will', 'those', 'being', 'haven', "mustn't", 'own', 'are', 'on', 'off', 'hasn', 'me', 'doing', 'do', 'himself', 'o', 'who', 'your', 'been', 'some', 'same', 're', 'don', 'can', "mightn't", 'below', 'he', 'only', "you've", 'the', 'yourselves', 'out', 'aren', 'because', 'hers', 'down', 'once', 'theirs', 'of', 'until', "couldn't", 'myself', "that'll", 'they', 'll', 'mightn', 'i', 'having', 'shan', "shouldn't", "weren't", "should've", 'you', 'other', "aren't", "wouldn't", 'ours', 'be', 'yourself', 'ma', 'in', "shan't", 'few', 'wasn', 'her', "isn't", 'between', 'but', 'very', 'just', 'too', 'couldn', 'each', 'our', 'herself', 'these', 'itself', 'ain', "you'll", 'needn', 'does', 'more', 'did', 'a', 'his', 'further', 'him', 'nor', "hadn't", 'd', "wasn't", 'have', 'now', 'under', 'by', 'after', 'm', 'against', 'which', "haven't", 'wouldn', 'that', "don't", "you're", 'were', 'through', 'should', 'not', 'bot

In [40]:
#normalized train reviews
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews[0]

'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

In [41]:
#Normalized test reviews
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[40000]

'first want say lean liber polit scale found movi offens manag watch whole doggon disgrac film thi movi bring low origin idea ye wa origin thu 2 star instead 1 film writer uncr onli come thi act wa horribl charact unlik part lead ladi stori good qualiti made bf sort bad guy see mayb miss someth knowh wa earth relev charact movi shell ani money thi garbag almost wish peta would come rescu thi aw offens movi form protest disgust say anymor'

Bags of words model

It is used to convert text documents to numerical vectors or bag of words.

In [42]:
#Count vectorizer for bag of words
cv=CountVectorizer()
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 156136)
BOW_cv_test: (10000, 156136)


Term Frequency-Inverse Document Frequency model (TFIDF)

It is used to convert text documents to matrix of tfidf features.

In [43]:
#Tfidf vectorizer
tv=TfidfVectorizer()
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 156136)
Tfidf_test: (10000, 156136)


In [44]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


Multinomial Naive Bayes for bag of words and tfidf features

In [45]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


Model performance on test data

In [46]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']
['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [47]:
#Accuracy of the model
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.8532
mnb_tfidf_score : 0.8626


In [48]:
#Classification report for bag of words
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.82      0.91      0.86      4993
    Negative       0.90      0.79      0.84      5007

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000

              precision    recall  f1-score   support

    Positive       0.85      0.88      0.86      4993
    Negative       0.88      0.85      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [52]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_20newsgroups
newsgroups_train=fetch_20newsgroups(subset='train',shuffle=True)
newsgroups_test=fetch_20newsgroups(subset='test',shuffle=True)


In [53]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [54]:
print(newsgroups_train.DESCR)


.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [57]:
newsgroups_train.data[:3]


["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [58]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(300)

In [59]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [61]:
print(WordNetLemmatizer().lemmatize('sent',pos='v'))

send


In [63]:

import pandas as pd
stemmer=SnowballStemmer('english')
original_words=['caresses','flies','dies','mules','denied','died','agreed','owned',
 'humbled','sized','meeting','stating','siezing','itemization','sensational',
 'traditional','reference','colonizer','plotted']
singles=[stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'Original word':original_words,'stemmed':singles})

,Original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [71]:
def lemmatize_stemming(text):
 return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))
def preprocess(text):
 result=[]
 for token in gensim.utils.simple_preprocess(text):
  if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
    result.append(lemmatize_stemming(token))

 return result

In [74]:
document=50
doc_sample='This disk had faild many times.I would like to get it replaced'
print('Original Document: ')
words=[]
for word in doc_sample.split(' '):
  words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(preprocess(doc_sample))

Original Document: 
['This', 'disk', 'had', 'faild', 'many', 'times.I', 'would', 'like', 'to', 'get', 'it', 'replaced']


 Tokenized and lemmatized document: 
['disk', 'faild', 'time', 'like', 'replac']


In [76]:
processed_docs=[]
for doc in newsgroups_train.data:
  processed_docs.append(preprocess(doc))
print(processed_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [77]:

dictionary=gensim.corpora.Dictionary(processed_docs)

In [79]:
count =0
for k,v in dictionary.iteritems():
 print(k,v)
 count+=1
 if count >15:
   break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten
11 funki
12 histori
13 host
14 info
15 know


In [80]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [82]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
document_num=25
bow_doc_x=bow_corpus[document_num]
for i in range(len(bow_doc_x)):
 print('Word {} (\"{}\") appears {} time:'.format(bow_doc_x[i][0],
 dictionary[bow_doc_x[i][0]],
 bow_doc_x[i][1]))


Word 19 ("separ") appears 1 time:
Word 38 ("experi") appears 1 time:
Word 63 ("summari") appears 1 time:
Word 67 ("access") appears 1 time:
Word 83 ("email") appears 1 time:
Word 91 ("impress") appears 1 time:
Word 154 ("system") appears 3 time:
Word 221 ("easili") appears 1 time:
Word 292 ("respond") appears 1 time:
Word 322 ("interfac") appears 1 time:
Word 354 ("appreci") appears 1 time:
Word 543 ("capabl") appears 1 time:
Word 563 ("develop") appears 1 time:
Word 568 ("environ") appears 1 time:
Word 690 ("offer") appears 1 time:
Word 693 ("purchas") appears 1 time:
Word 746 ("applic") appears 1 time:
Word 779 ("specif") appears 1 time:
Word 807 ("care") appears 1 time:
Word 875 ("maintain") appears 1 time:
Word 967 ("suffici") appears 1 time:
Word 987 ("unix") appears 2 time:
Word 1089 ("laboratori") appears 1 time:
Word 1092 ("adress") appears 1 time:
Word 1093 ("avoid") appears 1 time:
Word 1094 ("hassl") appears 1 time:
Word 1095 ("hello") appears 1 time:
Word 1096 ("input") app

In [83]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=8,id2word=dictionary,
 passes=10)

In [84]:
for idx,topic in lda_model.print_topics(-1):
 print('Topic: {} \n Words: {}'.format(idx,topic))
 print('\n')

Topic: 0 
 Words: 0.006*"washington" + 0.006*"health" + 0.006*"pitt" + 0.005*"medic" + 0.005*"food" + 0.005*"appl" + 0.005*"diseas" + 0.005*"caus" + 0.005*"bank" + 0.005*"netcom"


Topic: 1 
 Words: 0.009*"sale" + 0.009*"bike" + 0.009*"entri" + 0.008*"price" + 0.007*"uiuc" + 0.006*"cwru" + 0.006*"andrew" + 0.006*"sell" + 0.005*"car" + 0.005*"cleveland"


Topic: 2 
 Words: 0.021*"drive" + 0.009*"power" + 0.008*"scsi" + 0.006*"control" + 0.006*"wire" + 0.005*"hard" + 0.005*"engin" + 0.004*"build" + 0.004*"high" + 0.004*"light"


Topic: 3 
 Words: 0.014*"file" + 0.009*"window" + 0.009*"card" + 0.009*"chip" + 0.007*"encrypt" + 0.007*"program" + 0.006*"softwar" + 0.006*"version" + 0.006*"data" + 0.005*"disk"


Topic: 4 
 Words: 0.018*"game" + 0.016*"team" + 0.012*"play" + 0.010*"player" + 0.007*"hockey" + 0.006*"season" + 0.005*"score" + 0.005*"leagu" + 0.004*"basebal" + 0.004*"divis"


Topic: 5 
 Words: 0.014*"space" + 0.012*"window" + 0.010*"nasa" + 0.008*"program" + 0.006*"imag" + 0.005*